In [2]:
import pandas as pd
import random

In [3]:
import numpy as np

# DATASET DESCRIPTION

The data source for our Postgres database will be CSV files. 
Consequently, we need to create synthetic data in the required format. 
The dataset must include the following columns:
  'DiscountPercentage',
  'DiscountName',
  'DeliveryCompanyID',
  'DeliveryCompany',
  'ProductID' ,
  'ProductName',
  'StoreType',
  'PostalCode',
  'StoreCode',
  'StoreAddress',
  'InvoiceNo',
  'DeliveryType',
  'DiscountID',
  'Description',
  'Quantity',
  'InvoiceDate',
  'UnitPrice',
  'CustomerID',
  'Country',
  'StoreCity',
  'StoreRegion',
  'CustomerName',
  'CustomerSurname',
  'CustomerBirthday',
  'CustomerGender',
  'CustomerPhone',
  'CustomerEmail',
  'CustomerIncome',
  'ProductCategory',
  'ProductSubcategory',
  'Brand',
  'SupplierID',
  'Supplier',
  'DeliveryID',
  'CourierID',
  'CourierName',
  'CourierPhone',
  'DiscountDescription',
  'DiscountLimit',
  'DiscountAdd',
  'InvoiceLine',
  'LineCost',
  'TotalQuantity',
  'TotalAmount',
  'TotalAmount_After_Discount

The dataset provide a comprehensive overview of hypermarket sales, 
allowing for analysis and exploration of sales performance, customer 
preferences and behavior, the number of sales and profits.

We will create DWH for the sales business process and the fact table will 
reflect it.
One sale corresponds to one transaction(payment=invoice). 
Accordingly one sale has one invoice which may contain several products (lines) 
with different quantities.
Because of that the grain of the fact table is a line item in the invoice.

We also seek comprehensive details concerning the retail outlet of each transaction, 
client particulars, product information, delivery services, and discount structures. 
This data will prove instrumental in the analysis of diverse patterns, thereby 
enhancing our efficacy in formulating and optimizing sales strategies. Key areas of 
focus include:
- Profit calculations
- Identification of our primary clientele, encompassing demographic factors such as age, 
income level, and gender.
- Analysis of sales patterns, considering peak times and conditions of heightened activity.
- Assessment of our top-selling products with the highest profitability, as well as 
identification of less sought-after items.
- Evaluation of the most lucrative sales outlets within our network.
- Examination of the preferred delivery companies to inform optimal collaborative partnerships.

These insights will serve as valuable inputs for strategic decision-making and performance 
enhancement.

In [4]:
np.random.seed(42)

In [5]:
# generate 10000 unique IDs for InvoiceNo column
num_invoices = 10000
unique_invoices = np.arange(1, num_invoices + 1)

In [6]:
# shuffle the array to randomize the order
np.random.shuffle(unique_invoices)

In [7]:
# create a dataframe
df = pd.DataFrame({'InvoiceNo': unique_invoices})
df.head()

,InvoiceNo
0,6253
1,4685
2,1732
3,4743
4,4522


In [8]:
# check uniqueness of InvoiceNo column
invno_unique_count = df['InvoiceNo'].nunique()
print(f'The "InvoiceNo" column has {invno_unique_count} unique IDs.')

The "InvoiceNo" column has 10000 unique IDs.


In [9]:
# create 10000 unique dates for each particular InvoiceNo
# create variable which contains necessary numrer of rows
num_dates = len(df['InvoiceNo'].unique())

# Generate random timestamp values within the specified date range
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2023-11-30')
inv_dates = pd.to_datetime(np.random.uniform(start_date.timestamp(), end_date.timestamp(), size=num_dates), unit='s')

# Round the milliseconds part
inv_dates = inv_dates.round('1s') 

inv_dates

DatetimeIndex(['2023-01-18 23:32:43', '2020-02-02 01:55:10',
               '2022-05-07 05:40:59', '2022-09-20 17:48:16',
               '2023-03-25 13:55:07', '2020-03-21 00:39:19',
               '2021-11-17 01:46:41', '2022-02-04 01:04:46',
               '2020-08-29 16:18:23', '2021-04-05 18:44:19',
               ...
               '2023-03-04 10:48:32', '2023-05-04 20:56:02',
               '2022-10-01 12:20:56', '2020-05-12 19:43:35',
               '2020-01-23 23:28:58', '2020-08-28 20:55:21',
               '2020-06-17 18:14:47', '2022-11-02 05:19:17',
               '2022-03-06 23:06:21', '2022-01-21 14:00:39'],
              dtype='datetime64[ns]', length=10000, freq=None)

In [10]:
# Create a DataFrame from the DatetimeIndex 
dates_df = pd.DataFrame({'InvoiceDate': inv_dates})

In [11]:
# Add the InvoiceDate column to the original DataFrame using pd.concat
df1 = pd.concat([df, dates_df], axis=1)

# Print the resulting DataFrame
df1.head()

,InvoiceNo,InvoiceDate
0,6253,2023-01-18 23:32:43
1,4685,2020-02-02 01:55:10
2,1732,2022-05-07 05:40:59
3,4743,2022-09-20 17:48:16
4,4522,2023-03-25 13:55:07


In [12]:
# Remove duplicates based on all columns
df1 = df1.drop_duplicates()


In [13]:
# check that we have 10000 rows
df1_row_count = len(df1)
print(df1_row_count)

10000


In [14]:
# add customer info from csv file
#(we have 1000 customers)

In [15]:
df_cust = pd.read_csv('..DATASETS/customers.csv')
df_cust

,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail
0,1,Beret,Quin,3/8/1976,Female,487-791-8229,bquin0@mediafire.com
1,2,Orson,Godwin,2/15/1964,Male,407-335-6133,ogodwin1@seesaa.net
2,3,Kevon,Doxey,11/17/1986,Male,382-844-3080,kdoxey2@odnoklassniki.ru
3,4,Melba,Dryden,6/22/1971,Female,503-981-8336,mdryden3@drupal.org
4,5,Fredrika,de Glanville,12/11/1984,Female,160-797-9249,fdeglanville4@economist.com
...,...,...,...,...,...,...,...
995,996,Benedetta,Clemot,3/29/1994,Female,902-321-7300,bclemotrn@skype.com
996,997,Kacie,Lettson,7/4/1996,Female,437-235-9331,klettsonro@networkadvertising.org
997,998,Binni,Lukehurst,12/2/1999,Female,203-139-2190,blukehurstrp@amazon.com
998,999,Chrisse,Please,4/18/1990,Male,112-493-2053,cpleaserq@histats.com


In [16]:
# add new column 'CustomerIncome' to existing customers df
# create variable with possible values
cust_income = ['less 25000', '26000-50000', 'more than 50000']

In [17]:
df_cust['CustomerIncome'] = np.random.choice(cust_income, size=len(df_cust))
df_cust

,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome
0,1,Beret,Quin,3/8/1976,Female,487-791-8229,bquin0@mediafire.com,26000-50000
1,2,Orson,Godwin,2/15/1964,Male,407-335-6133,ogodwin1@seesaa.net,less 25000
2,3,Kevon,Doxey,11/17/1986,Male,382-844-3080,kdoxey2@odnoklassniki.ru,26000-50000
3,4,Melba,Dryden,6/22/1971,Female,503-981-8336,mdryden3@drupal.org,less 25000
4,5,Fredrika,de Glanville,12/11/1984,Female,160-797-9249,fdeglanville4@economist.com,26000-50000
...,...,...,...,...,...,...,...,...
995,996,Benedetta,Clemot,3/29/1994,Female,902-321-7300,bclemotrn@skype.com,26000-50000
996,997,Kacie,Lettson,7/4/1996,Female,437-235-9331,klettsonro@networkadvertising.org,less 25000
997,998,Binni,Lukehurst,12/2/1999,Female,203-139-2190,blukehurstrp@amazon.com,26000-50000
998,999,Chrisse,Please,4/18/1990,Male,112-493-2053,cpleaserq@histats.com,more than 50000


In [18]:
df_cust['CustomerBirthday'] = pd.to_datetime(df_cust['CustomerBirthday'])

# Remove the time part by using .dt.date to get only the date
df_cust['CustomerBirthday'] = df_cust['CustomerBirthday'].dt.date

In [19]:
# Randomly select rows from df1 to match the size of df1
df_cust1 = df_cust.sample(n=len(df1), replace=True)

# Concatenate df1 with the selected rows from df1
df2 = pd.concat([df1, df_cust1.reset_index(drop=True)], axis=1)

df2

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome
0,6253,2023-01-18 23:32:43,856,Horace,Crocket,1975-02-09,Male,135-942-6936,hcrocketnr@wunderground.com,less 25000
1,4685,2020-02-02 01:55:10,56,Dorthea,Brewster,1969-12-13,Female,925-496-5933,dbrewster1j@about.com,26000-50000
2,1732,2022-05-07 05:40:59,207,Darcie,Laidel,1999-07-07,Agender,750-497-1391,dlaidel5q@over-blog.com,26000-50000
3,4743,2022-09-20 17:48:16,105,Morey,Tincey,1992-05-22,Male,549-717-4109,mtincey2w@sina.com.cn,less 25000
4,4522,2023-03-25 13:55:07,547,Dinah,Popland,1995-10-06,Female,796-135-4782,dpoplandf6@zimbio.com,less 25000
...,...,...,...,...,...,...,...,...,...,...
9995,5735,2020-08-28 20:55:21,937,Jarad,Watsham,1993-06-19,Male,793-530-2724,jwatshamq0@wikipedia.org,26000-50000
9996,5192,2020-06-17 18:14:47,601,Dyann,Lawrie,1970-05-03,Female,382-826-7050,dlawriego@washington.edu,more than 50000
9997,5391,2022-11-02 05:19:17,607,Barty,Blodgett,1972-05-20,Male,945-792-5398,bblodgettgu@kickstarter.com,more than 50000
9998,861,2022-03-06 23:06:21,887,Livy,Chazier,1992-07-02,Female,227-914-8662,lchazierom@shinystat.com,less 25000


In [20]:
un_count = df2['CustomerID'].nunique()
print(un_count)

1000


In [21]:
un_count1 = df2['CustomerID'].count()
print(un_count1)

10000


In [22]:
# add discount information to each InvoiceNo
# we have 30 discount programs
d_id = 30
discount_program = np.arange(1, d_id + 1)

np.random.shuffle(unique_invoices)

disc_program = pd.DataFrame({'DiscountID': discount_program})

print(disc_program)

    DiscountID
0            1
1            2
2            3
3            4
4            5
5            6
6            7
7            8
8            9
9           10
10          11
11          12
12          13
13          14
14          15
15          16
16          17
17          18
18          19
19          20
20          21
21          22
22          23
23          24
24          25
25          26
26          27
27          28
28          29
29          30


In [23]:
# List of 30 unique discount names
discount_names = [
    'Happy Hours',
    'Black Friday',
    'Flash Sale',
    'Clearance',
    'Early Bird Special',
    'Cyber Monday',
    'BOGO (Buy One Get One)',
    'Weekend Special',
    'Holiday Discount',
    'Limited Time Offer',
    'Student Discount',
    'Senior Citizen Discount',
    'Military Discount',
    'Employee Discount',
    'VIP Sale',
    'Spring Sale',
    'Summer Clearance',
    'Fall Frenzy',
    'Winter Wonderland',
    'Grand Opening Sale',
    'Anniversary Sale',
    'Back-to-School Sale',
    'Super Savings',
    'Final Markdowns',
    'Member Exclusive',
    'Bundle and Save',
    'Family and Friends Event',
    'Customer Appreciation',
    'Double Discount Day'
]

In [24]:
disc_program['DiscountName'] = np.random.choice(discount_names, size=len(disc_program))
print(disc_program)

    DiscountID             DiscountName
0            1              Spring Sale
1            2              Spring Sale
2            3              Spring Sale
3            4             Cyber Monday
4            5                 VIP Sale
5            6               Flash Sale
6            7         Holiday Discount
7            8              Happy Hours
8            9         Student Discount
9           10      Back-to-School Sale
10          11             Cyber Monday
11          12          Final Markdowns
12          13       Early Bird Special
13          14         Member Exclusive
14          15         Member Exclusive
15          16    Customer Appreciation
16          17              Fall Frenzy
17          18         Anniversary Sale
18          19  Senior Citizen Discount
19          20   BOGO (Buy One Get One)
20          21             Black Friday
21          22   BOGO (Buy One Get One)
22          23         Holiday Discount
23          24          Final Markdowns


In [25]:
disc_program['DiscountPercentage'] = np.random.randint(3,13, size=len(disc_program))
print(disc_program)

    DiscountID             DiscountName  DiscountPercentage
0            1              Spring Sale                   4
1            2              Spring Sale                  12
2            3              Spring Sale                   5
3            4             Cyber Monday                   3
4            5                 VIP Sale                   3
5            6               Flash Sale                  12
6            7         Holiday Discount                   8
7            8              Happy Hours                   3
8            9         Student Discount                   7
9           10      Back-to-School Sale                   8
10          11             Cyber Monday                  11
11          12          Final Markdowns                   8
12          13       Early Bird Special                   6
13          14         Member Exclusive                   3
14          15         Member Exclusive                   8
15          16    Customer Appreciation 

In [26]:

discount_descriptions = [
    'Enjoy special pricing during specific hours of the day.',
    'Massive discounts on the day after Thanksgiving.',
    'Limited-time sale with quick and steep discounts.',
    'Clearing out inventory with significant price reductions.',
    'Special deals for early shoppers.',
    'Discounts specifically for the weekend.',
    'Special pricing for holiday seasons.',
    'Offer available for a short period.',
    'Exclusive sale for VIP customers.',
    'Special pricing for the spring season.',
    'Clearance sale for summer products.',
    'Special deals for the fall season.',
    'Winter-themed discounts and promotions.',
    'Discounts for back-to-school shopping.',
    'Big savings on select items.',
    'Markdowns on final products.',
    'Exclusive discounts for members only.',
    'Save money by bundling items together.',
    'Special event for family and friends with discounts.',
    'Appreciation event for loyal customers.',
    'Day with double the usual discount rate.'
]

In [27]:
disc_program['DiscountDescription'] = np.random.choice(discount_descriptions, size=len(disc_program))
print(disc_program)

    DiscountID             DiscountName  DiscountPercentage  \
0            1              Spring Sale                   4   
1            2              Spring Sale                  12   
2            3              Spring Sale                   5   
3            4             Cyber Monday                   3   
4            5                 VIP Sale                   3   
5            6               Flash Sale                  12   
6            7         Holiday Discount                   8   
7            8              Happy Hours                   3   
8            9         Student Discount                   7   
9           10      Back-to-School Sale                   8   
10          11             Cyber Monday                  11   
11          12          Final Markdowns                   8   
12          13       Early Bird Special                   6   
13          14         Member Exclusive                   3   
14          15         Member Exclusive                

In [28]:
discount_limits = [
    'Permanent',
    'Temporal',
    'Only work days',
    'Only morning hours'
]

In [29]:
disc_program['DiscountLimit'] = np.random.choice(discount_limits, size=len(disc_program))
print(disc_program)

    DiscountID             DiscountName  DiscountPercentage  \
0            1              Spring Sale                   4   
1            2              Spring Sale                  12   
2            3              Spring Sale                   5   
3            4             Cyber Monday                   3   
4            5                 VIP Sale                   3   
5            6               Flash Sale                  12   
6            7         Holiday Discount                   8   
7            8              Happy Hours                   3   
8            9         Student Discount                   7   
9           10      Back-to-School Sale                   8   
10          11             Cyber Monday                  11   
11          12          Final Markdowns                   8   
12          13       Early Bird Special                   6   
13          14         Member Exclusive                   3   
14          15         Member Exclusive                

In [30]:
additional_info = [
    'Valid only for items at regular prices.',
    'Applicable to in-store purchases only.',
    'Online exclusive discounts.',
    'Limited quantity available.',
    'Not applicable to clearance items.',
    'One-time use per customer.',
    'Cannot be combined with other offers.',
    'Minimum purchase requirement applies.',
    'Discount applies at checkout.',
    'Excludes certain product categories.'
]

In [31]:
disc_program['DiscountAdd'] = np.random.choice(additional_info, size=len(disc_program))
print(disc_program)

    DiscountID             DiscountName  DiscountPercentage  \
0            1              Spring Sale                   4   
1            2              Spring Sale                  12   
2            3              Spring Sale                   5   
3            4             Cyber Monday                   3   
4            5                 VIP Sale                   3   
5            6               Flash Sale                  12   
6            7         Holiday Discount                   8   
7            8              Happy Hours                   3   
8            9         Student Discount                   7   
9           10      Back-to-School Sale                   8   
10          11             Cyber Monday                  11   
11          12          Final Markdowns                   8   
12          13       Early Bird Special                   6   
13          14         Member Exclusive                   3   
14          15         Member Exclusive                

In [32]:
# add discounts info to main df
# Randomly select rows from df1 to match the size of df1
disc_program1 = disc_program.sample(n=len(df2), replace=True)

# Concatenate df1 with the selected rows from df1
df3 = pd.concat([df2, disc_program1.reset_index(drop=True)], axis=1)

df3

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,DiscountID,DiscountName,DiscountPercentage,DiscountDescription,DiscountLimit,DiscountAdd
0,6253,2023-01-18 23:32:43,856,Horace,Crocket,1975-02-09,Male,135-942-6936,hcrocketnr@wunderground.com,less 25000,8,Happy Hours,3,Special event for family and friends with disc...,Temporal,Excludes certain product categories.
1,4685,2020-02-02 01:55:10,56,Dorthea,Brewster,1969-12-13,Female,925-496-5933,dbrewster1j@about.com,26000-50000,6,Flash Sale,12,Limited-time sale with quick and steep discounts.,Only morning hours,Excludes certain product categories.
2,1732,2022-05-07 05:40:59,207,Darcie,Laidel,1999-07-07,Agender,750-497-1391,dlaidel5q@over-blog.com,26000-50000,15,Member Exclusive,8,Special event for family and friends with disc...,Temporal,Valid only for items at regular prices.
3,4743,2022-09-20 17:48:16,105,Morey,Tincey,1992-05-22,Male,549-717-4109,mtincey2w@sina.com.cn,less 25000,25,Bundle and Save,10,Markdowns on final products.,Only morning hours,Online exclusive discounts.
4,4522,2023-03-25 13:55:07,547,Dinah,Popland,1995-10-06,Female,796-135-4782,dpoplandf6@zimbio.com,less 25000,3,Spring Sale,5,Exclusive sale for VIP customers.,Temporal,Not applicable to clearance items.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5735,2020-08-28 20:55:21,937,Jarad,Watsham,1993-06-19,Male,793-530-2724,jwatshamq0@wikipedia.org,26000-50000,24,Final Markdowns,4,Winter-themed discounts and promotions.,Temporal,Applicable to in-store purchases only.
9996,5192,2020-06-17 18:14:47,601,Dyann,Lawrie,1970-05-03,Female,382-826-7050,dlawriego@washington.edu,more than 50000,7,Holiday Discount,8,Massive discounts on the day after Thanksgiving.,Only morning hours,Valid only for items at regular prices.
9997,5391,2022-11-02 05:19:17,607,Barty,Blodgett,1972-05-20,Male,945-792-5398,bblodgettgu@kickstarter.com,more than 50000,19,Senior Citizen Discount,12,Enjoy special pricing during specific hours of...,Only morning hours,Excludes certain product categories.
9998,861,2022-03-06 23:06:21,887,Livy,Chazier,1992-07-02,Female,227-914-8662,lchazierom@shinystat.com,less 25000,7,Holiday Discount,8,Massive discounts on the day after Thanksgiving.,Only morning hours,Valid only for items at regular prices.


In [33]:
un_count1 = df3['DiscountID'].count()
print(un_count1)

10000


In [34]:
un_count1 = df3['DiscountID'].nunique()
print(un_count1)

30


In [35]:
# add delivery type for each invoice
delivery_types = ['home standart', 'home express', 'offline purchase', 'pick point', 'pick up from the store']

df3['DeliveryType'] = np.random.choice(delivery_types, size=len(df3))
df3

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,DiscountID,DiscountName,DiscountPercentage,DiscountDescription,DiscountLimit,DiscountAdd,DeliveryType
0,6253,2023-01-18 23:32:43,856,Horace,Crocket,1975-02-09,Male,135-942-6936,hcrocketnr@wunderground.com,less 25000,8,Happy Hours,3,Special event for family and friends with disc...,Temporal,Excludes certain product categories.,pick up from the store
1,4685,2020-02-02 01:55:10,56,Dorthea,Brewster,1969-12-13,Female,925-496-5933,dbrewster1j@about.com,26000-50000,6,Flash Sale,12,Limited-time sale with quick and steep discounts.,Only morning hours,Excludes certain product categories.,offline purchase
2,1732,2022-05-07 05:40:59,207,Darcie,Laidel,1999-07-07,Agender,750-497-1391,dlaidel5q@over-blog.com,26000-50000,15,Member Exclusive,8,Special event for family and friends with disc...,Temporal,Valid only for items at regular prices.,pick up from the store
3,4743,2022-09-20 17:48:16,105,Morey,Tincey,1992-05-22,Male,549-717-4109,mtincey2w@sina.com.cn,less 25000,25,Bundle and Save,10,Markdowns on final products.,Only morning hours,Online exclusive discounts.,offline purchase
4,4522,2023-03-25 13:55:07,547,Dinah,Popland,1995-10-06,Female,796-135-4782,dpoplandf6@zimbio.com,less 25000,3,Spring Sale,5,Exclusive sale for VIP customers.,Temporal,Not applicable to clearance items.,home standart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5735,2020-08-28 20:55:21,937,Jarad,Watsham,1993-06-19,Male,793-530-2724,jwatshamq0@wikipedia.org,26000-50000,24,Final Markdowns,4,Winter-themed discounts and promotions.,Temporal,Applicable to in-store purchases only.,home express
9996,5192,2020-06-17 18:14:47,601,Dyann,Lawrie,1970-05-03,Female,382-826-7050,dlawriego@washington.edu,more than 50000,7,Holiday Discount,8,Massive discounts on the day after Thanksgiving.,Only morning hours,Valid only for items at regular prices.,pick up from the store
9997,5391,2022-11-02 05:19:17,607,Barty,Blodgett,1972-05-20,Male,945-792-5398,bblodgettgu@kickstarter.com,more than 50000,19,Senior Citizen Discount,12,Enjoy special pricing during specific hours of...,Only morning hours,Excludes certain product categories.,offline purchase
9998,861,2022-03-06 23:06:21,887,Livy,Chazier,1992-07-02,Female,227-914-8662,lchazierom@shinystat.com,less 25000,7,Holiday Discount,8,Massive discounts on the day after Thanksgiving.,Only morning hours,Valid only for items at regular prices.,home express


In [36]:
# add DeliveryId column - unique for each InvoiceNo
def generate_delivery_id():
    letter = ''.join(random.choice(string.ascii_uppercase) for _ in range(3))
    digits = ''.join(random.choice(string.digits) for _ in range(6))
    letter = ''.join(random.choice(string.ascii_lowercase) for _ in range(2))
    return letter + digits 



In [37]:
import string

df3['DeliveryID'] = [generate_delivery_id() for _ in range(len(df3))]
df3

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,DiscountID,DiscountName,DiscountPercentage,DiscountDescription,DiscountLimit,DiscountAdd,DeliveryType,DeliveryID
0,6253,2023-01-18 23:32:43,856,Horace,Crocket,1975-02-09,Male,135-942-6936,hcrocketnr@wunderground.com,less 25000,8,Happy Hours,3,Special event for family and friends with disc...,Temporal,Excludes certain product categories.,pick up from the store,il104757
1,4685,2020-02-02 01:55:10,56,Dorthea,Brewster,1969-12-13,Female,925-496-5933,dbrewster1j@about.com,26000-50000,6,Flash Sale,12,Limited-time sale with quick and steep discounts.,Only morning hours,Excludes certain product categories.,offline purchase,sw836103
2,1732,2022-05-07 05:40:59,207,Darcie,Laidel,1999-07-07,Agender,750-497-1391,dlaidel5q@over-blog.com,26000-50000,15,Member Exclusive,8,Special event for family and friends with disc...,Temporal,Valid only for items at regular prices.,pick up from the store,uf592497
3,4743,2022-09-20 17:48:16,105,Morey,Tincey,1992-05-22,Male,549-717-4109,mtincey2w@sina.com.cn,less 25000,25,Bundle and Save,10,Markdowns on final products.,Only morning hours,Online exclusive discounts.,offline purchase,mu510169
4,4522,2023-03-25 13:55:07,547,Dinah,Popland,1995-10-06,Female,796-135-4782,dpoplandf6@zimbio.com,less 25000,3,Spring Sale,5,Exclusive sale for VIP customers.,Temporal,Not applicable to clearance items.,home standart,if039917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5735,2020-08-28 20:55:21,937,Jarad,Watsham,1993-06-19,Male,793-530-2724,jwatshamq0@wikipedia.org,26000-50000,24,Final Markdowns,4,Winter-themed discounts and promotions.,Temporal,Applicable to in-store purchases only.,home express,pt331517
9996,5192,2020-06-17 18:14:47,601,Dyann,Lawrie,1970-05-03,Female,382-826-7050,dlawriego@washington.edu,more than 50000,7,Holiday Discount,8,Massive discounts on the day after Thanksgiving.,Only morning hours,Valid only for items at regular prices.,pick up from the store,gx646423
9997,5391,2022-11-02 05:19:17,607,Barty,Blodgett,1972-05-20,Male,945-792-5398,bblodgettgu@kickstarter.com,more than 50000,19,Senior Citizen Discount,12,Enjoy special pricing during specific hours of...,Only morning hours,Excludes certain product categories.,offline purchase,xo627715
9998,861,2022-03-06 23:06:21,887,Livy,Chazier,1992-07-02,Female,227-914-8662,lchazierom@shinystat.com,less 25000,7,Holiday Discount,8,Massive discounts on the day after Thanksgiving.,Only morning hours,Valid only for items at regular prices.,home express,cr312601


In [38]:
unique_del_id = df3['DeliveryID'].nunique()
print(unique_del_id)

10000


In [39]:
# add Stores information

stores_code = [
    'WE6587-g',
    'WE6587-d',
    'WE6527-f',
    'WY6587-g',
    'TE6587-d',
    'LE6527-f',
    'GK0587-g',
    'WO6587-d',
    'PE6527-f',
    'WY6587-q',
    'JE7887-d',
    'ZE6527-r'
]

stores = pd.DataFrame({'StoreCode': stores_code})
print(stores)

   StoreCode
0   WE6587-g
1   WE6587-d
2   WE6527-f
3   WY6587-g
4   TE6587-d
5   LE6527-f
6   GK0587-g
7   WO6587-d
8   PE6527-f
9   WY6587-q
10  JE7887-d
11  ZE6527-r


In [40]:
store_type = [
    'online',
    'direct'
]

stores['StoreType'] = np.random.choice(store_type, size=len(stores))
print(stores)

   StoreCode StoreType
0   WE6587-g    online
1   WE6587-d    online
2   WE6527-f    online
3   WY6587-g    online
4   TE6587-d    online
5   LE6527-f    direct
6   GK0587-g    direct
7   WO6587-d    online
8   PE6527-f    online
9   WY6587-q    online
10  JE7887-d    online
11  ZE6527-r    direct


In [41]:
stores['PostalCode'] = np.random.randint(188362, 956372, size=len(stores))
print(stores)

   StoreCode StoreType  PostalCode
0   WE6587-g    online      248559
1   WE6587-d    online      821235
2   WE6527-f    online      757626
3   WY6587-g    online      261048
4   TE6587-d    online      623103
5   LE6527-f    direct      806129
6   GK0587-g    direct      541002
7   WO6587-d    online      890689
8   PE6527-f    online      663477
9   WY6587-q    online      208002
10  JE7887-d    online      285078
11  ZE6527-r    direct      867618


In [42]:
map = {
    'Country': ['UK', 'UK', 'Spain', 'Spain'],
    'StoreRegion': ['London', 'East of England', 'Andalucia', 'Madrid'],
    'StoreCity': ['London', 'Luton', 'Malaga', 'Madrid']
}

location = pd.DataFrame(map)
location

,Country,StoreRegion,StoreCity
0,UK,London,London
1,UK,East of England,Luton
2,Spain,Andalucia,Malaga
3,Spain,Madrid,Madrid


In [43]:
loc = location.sample(n=len(stores), replace=True)

# Concatenate df1 with the selected rows from df1
stores1 = pd.concat([stores, loc.reset_index(drop=True)], axis=1)

stores1

,StoreCode,StoreType,PostalCode,Country,StoreRegion,StoreCity
0,WE6587-g,online,248559,UK,East of England,Luton
1,WE6587-d,online,821235,UK,East of England,Luton
2,WE6527-f,online,757626,Spain,Madrid,Madrid
3,WY6587-g,online,261048,UK,East of England,Luton
4,TE6587-d,online,623103,Spain,Andalucia,Malaga
5,LE6527-f,direct,806129,UK,London,London
6,GK0587-g,direct,541002,Spain,Andalucia,Malaga
7,WO6587-d,online,890689,UK,London,London
8,PE6527-f,online,663477,Spain,Madrid,Madrid
9,WY6587-q,online,208002,UK,London,London


In [44]:
add =[
    'Canona, Follard street, 64',
    'East Rogerberg, Cynthia Loaf, 566',
    'Cagona, Ron street, 6',
    'East Rogerberg, Gentk Road, 576',
    'New Town, Elm Street, 123',
    'West Side, Oak Avenue, 789',
    'Sunnyville, Maple Lane, 456',
    'Green Hills, Pine Road, 321',
    'Ocean View, Palm Street, 654',
    'Mountain Valley, Cedar Lane, 987',
    'Riverfront, Willow Avenue, 234',
    'City Center, Birch Road, 876'
]

addresses = pd.DataFrame({'StoreAddress': add})

stores2 = pd.concat([stores1, addresses], axis=1)
stores2


,StoreCode,StoreType,PostalCode,Country,StoreRegion,StoreCity,StoreAddress
0,WE6587-g,online,248559,UK,East of England,Luton,"Canona, Follard street, 64"
1,WE6587-d,online,821235,UK,East of England,Luton,"East Rogerberg, Cynthia Loaf, 566"
2,WE6527-f,online,757626,Spain,Madrid,Madrid,"Cagona, Ron street, 6"
3,WY6587-g,online,261048,UK,East of England,Luton,"East Rogerberg, Gentk Road, 576"
4,TE6587-d,online,623103,Spain,Andalucia,Malaga,"New Town, Elm Street, 123"
5,LE6527-f,direct,806129,UK,London,London,"West Side, Oak Avenue, 789"
6,GK0587-g,direct,541002,Spain,Andalucia,Malaga,"Sunnyville, Maple Lane, 456"
7,WO6587-d,online,890689,UK,London,London,"Green Hills, Pine Road, 321"
8,PE6527-f,online,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654"
9,WY6587-q,online,208002,UK,London,London,"Mountain Valley, Cedar Lane, 987"


In [45]:
stores2 = pd.concat([stores1, addresses], axis=1)
stores2

,StoreCode,StoreType,PostalCode,Country,StoreRegion,StoreCity,StoreAddress
0,WE6587-g,online,248559,UK,East of England,Luton,"Canona, Follard street, 64"
1,WE6587-d,online,821235,UK,East of England,Luton,"East Rogerberg, Cynthia Loaf, 566"
2,WE6527-f,online,757626,Spain,Madrid,Madrid,"Cagona, Ron street, 6"
3,WY6587-g,online,261048,UK,East of England,Luton,"East Rogerberg, Gentk Road, 576"
4,TE6587-d,online,623103,Spain,Andalucia,Malaga,"New Town, Elm Street, 123"
5,LE6527-f,direct,806129,UK,London,London,"West Side, Oak Avenue, 789"
6,GK0587-g,direct,541002,Spain,Andalucia,Malaga,"Sunnyville, Maple Lane, 456"
7,WO6587-d,online,890689,UK,London,London,"Green Hills, Pine Road, 321"
8,PE6527-f,online,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654"
9,WY6587-q,online,208002,UK,London,London,"Mountain Valley, Cedar Lane, 987"


In [46]:
# add stores to the main df
stores_repl = stores2.sample(n=len(df3), replace=True)

# Concatenate df1 with the selected rows from df1
df4 = pd.concat([df3, stores_repl.reset_index(drop=True)], axis=1)

df4

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,DiscountAdd,DeliveryType,DeliveryID,StoreCode,StoreType,PostalCode,Country,StoreRegion,StoreCity,StoreAddress
0,6253,2023-01-18 23:32:43,856,Horace,Crocket,1975-02-09,Male,135-942-6936,hcrocketnr@wunderground.com,less 25000,...,Excludes certain product categories.,pick up from the store,il104757,WE6587-g,online,248559,UK,East of England,Luton,"Canona, Follard street, 64"
1,4685,2020-02-02 01:55:10,56,Dorthea,Brewster,1969-12-13,Female,925-496-5933,dbrewster1j@about.com,26000-50000,...,Excludes certain product categories.,offline purchase,sw836103,WO6587-d,online,890689,UK,London,London,"Green Hills, Pine Road, 321"
2,1732,2022-05-07 05:40:59,207,Darcie,Laidel,1999-07-07,Agender,750-497-1391,dlaidel5q@over-blog.com,26000-50000,...,Valid only for items at regular prices.,pick up from the store,uf592497,WO6587-d,online,890689,UK,London,London,"Green Hills, Pine Road, 321"
3,4743,2022-09-20 17:48:16,105,Morey,Tincey,1992-05-22,Male,549-717-4109,mtincey2w@sina.com.cn,less 25000,...,Online exclusive discounts.,offline purchase,mu510169,ZE6527-r,direct,867618,Spain,Madrid,Madrid,"City Center, Birch Road, 876"
4,4522,2023-03-25 13:55:07,547,Dinah,Popland,1995-10-06,Female,796-135-4782,dpoplandf6@zimbio.com,less 25000,...,Not applicable to clearance items.,home standart,if039917,WY6587-q,online,208002,UK,London,London,"Mountain Valley, Cedar Lane, 987"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5735,2020-08-28 20:55:21,937,Jarad,Watsham,1993-06-19,Male,793-530-2724,jwatshamq0@wikipedia.org,26000-50000,...,Applicable to in-store purchases only.,home express,pt331517,PE6527-f,online,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654"
9996,5192,2020-06-17 18:14:47,601,Dyann,Lawrie,1970-05-03,Female,382-826-7050,dlawriego@washington.edu,more than 50000,...,Valid only for items at regular prices.,pick up from the store,gx646423,PE6527-f,online,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654"
9997,5391,2022-11-02 05:19:17,607,Barty,Blodgett,1972-05-20,Male,945-792-5398,bblodgettgu@kickstarter.com,more than 50000,...,Excludes certain product categories.,offline purchase,xo627715,TE6587-d,online,623103,Spain,Andalucia,Malaga,"New Town, Elm Street, 123"
9998,861,2022-03-06 23:06:21,887,Livy,Chazier,1992-07-02,Female,227-914-8662,lchazierom@shinystat.com,less 25000,...,Valid only for items at regular prices.,home express,cr312601,LE6527-f,direct,806129,UK,London,London,"West Side, Oak Avenue, 789"


In [47]:
# add delivery company and couriers info
# information about couriers we have in csv

couriers = pd.read_csv('..DATASETS/couriers.csv')
couriers

,CourierID,CourierName,CourierPhone
0,64605,Junette Felipe,732-195-2671
1,62798,Simone Stieger,449-333-1751
2,55041,Nana Vaudin,952-703-0922
3,39193,Allx Biddiss,188-845-3346
4,77609,Liliane Craney,427-548-3260
...,...,...,...
195,74379,Luz Lunt,171-783-8823
196,49966,Annalise Jankiewicz,834-225-6816
197,57924,Hardy Ravenscraft,720-593-9368
198,72666,Shawna Kemble,404-602-1245


In [48]:
# distribute companies accross couriers
del_comp = pd.read_csv('..DATASETS/delcompany.csv')
del_comp

,DeliveryCompanyID,DeliveryCompany
0,4889,Goyette Group
1,4534,O'Kon-Walsh
2,4921,Stracke-Jones
3,3870,Towne-Osinski
4,2475,Trantow and Sons
5,3319,Bosco-Hauck
6,3543,McDermott-Haag
7,3254,Schinner-Stoltenberg
8,3955,Brown-Padberg
9,2631,Gleason Group


In [49]:
del_comp1 = del_comp.sample(n=len(couriers), replace=True)

comp_cour = pd.concat([couriers, del_comp1.reset_index(drop=True)], axis=1)
comp_cour

,CourierID,CourierName,CourierPhone,DeliveryCompanyID,DeliveryCompany
0,64605,Junette Felipe,732-195-2671,4362,"Purdy, McCullough and Kuhn"
1,62798,Simone Stieger,449-333-1751,4332,Frami-Pfannerstill
2,55041,Nana Vaudin,952-703-0922,2678,Nitzsche-Gulgowski
3,39193,Allx Biddiss,188-845-3346,3543,McDermott-Haag
4,77609,Liliane Craney,427-548-3260,4536,"Carroll, Renner and Koelpin"
...,...,...,...,...,...
195,74379,Luz Lunt,171-783-8823,4469,Lakin Inc
196,49966,Annalise Jankiewicz,834-225-6816,4805,O'Kon-Leuschke
197,57924,Hardy Ravenscraft,720-593-9368,5447,Leuschke and Sons
198,72666,Shawna Kemble,404-602-1245,5473,Little LLC


In [50]:
comp_id_count = comp_cour['DeliveryCompanyID'].nunique()
print(comp_id_count)

30


In [51]:
comp_count = comp_cour['DeliveryCompany'].nunique()
print(comp_count)

30


In [52]:
# add company and couriers info to the main df
comp_cour1 = comp_cour.sample(n=len(df4), replace=True)

df5 = pd.concat([df4, comp_cour1.reset_index(drop=True)], axis=1)
df5


,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,PostalCode,Country,StoreRegion,StoreCity,StoreAddress,CourierID,CourierName,CourierPhone,DeliveryCompanyID,DeliveryCompany
0,6253,2023-01-18 23:32:43,856,Horace,Crocket,1975-02-09,Male,135-942-6936,hcrocketnr@wunderground.com,less 25000,...,248559,UK,East of England,Luton,"Canona, Follard street, 64",85991,Alisa Ivins,694-378-6329,2860,"Cummings, Mitchell and Reynolds"
1,4685,2020-02-02 01:55:10,56,Dorthea,Brewster,1969-12-13,Female,925-496-5933,dbrewster1j@about.com,26000-50000,...,890689,UK,London,London,"Green Hills, Pine Road, 321",54743,Andrea Melluish,432-853-7949,3747,"Schaden, Beahan and Vandervort"
2,1732,2022-05-07 05:40:59,207,Darcie,Laidel,1999-07-07,Agender,750-497-1391,dlaidel5q@over-blog.com,26000-50000,...,890689,UK,London,London,"Green Hills, Pine Road, 321",36371,Antoinette Rappaport,928-696-6516,4469,Lakin Inc
3,4743,2022-09-20 17:48:16,105,Morey,Tincey,1992-05-22,Male,549-717-4109,mtincey2w@sina.com.cn,less 25000,...,867618,Spain,Madrid,Madrid,"City Center, Birch Road, 876",50088,Patty Brocking,581-863-1371,4921,Stracke-Jones
4,4522,2023-03-25 13:55:07,547,Dinah,Popland,1995-10-06,Female,796-135-4782,dpoplandf6@zimbio.com,less 25000,...,208002,UK,London,London,"Mountain Valley, Cedar Lane, 987",56218,Wilhelmine Bennough,863-331-1608,4362,"Purdy, McCullough and Kuhn"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5735,2020-08-28 20:55:21,937,Jarad,Watsham,1993-06-19,Male,793-530-2724,jwatshamq0@wikipedia.org,26000-50000,...,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654",57924,Hardy Ravenscraft,720-593-9368,5447,Leuschke and Sons
9996,5192,2020-06-17 18:14:47,601,Dyann,Lawrie,1970-05-03,Female,382-826-7050,dlawriego@washington.edu,more than 50000,...,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654",23931,Hetti Jimmison,279-935-4171,4889,Goyette Group
9997,5391,2022-11-02 05:19:17,607,Barty,Blodgett,1972-05-20,Male,945-792-5398,bblodgettgu@kickstarter.com,more than 50000,...,623103,Spain,Andalucia,Malaga,"New Town, Elm Street, 123",64574,Cletis Littlekit,688-939-5620,3955,Brown-Padberg
9998,861,2022-03-06 23:06:21,887,Livy,Chazier,1992-07-02,Female,227-914-8662,lchazierom@shinystat.com,less 25000,...,806129,UK,London,London,"West Side, Oak Avenue, 789",24078,Editha Tamblyn,387-587-2856,5473,Little LLC


In [53]:
comp_id_count = df5['DeliveryCompanyID'].nunique()
print(comp_id_count)


30


In [54]:
cust_count = df5['CustomerID'].nunique()
print(cust_count)


1000


In [55]:
cust_phone_count = df5['CustomerPhone'].nunique()
print(cust_phone_count)


1000


In [56]:
# add info about Products

products = pd.read_csv('../DATASETS/products.csv')
products



,ProductName,Description,UnitPrice
0,Lamb Rack Frenched Australian,Muffin - Carrot Individual Wrap,$31.96
1,Wine - Ej Gallo Sierra Valley,Vanilla Beans,$41.80
2,Lid Coffee Cup 8oz Blk,Beer - Upper Canada Lager,$205.51
3,"Asparagus - White, Fresh",Spinach - Packaged,$37.50
4,"Mop Head - Cotton, 24 Oz",NaN,$213.22
...,...,...,...
295,Coffee Cup 12oz 5342cd,Bread - Raisin Walnut Oval,$94.98
296,Aromat Spice / Seasoning,Instant Coffee,$126.08
297,Cornstarch,Canadian Emmenthal,$199.63
298,"Napkin - Cocktail,beige 2 - Ply",Chevril,$133.28


In [57]:
# ProductId column contain the same ID for all products
# generate new IDs
products['ProductID'] = np.random.randint(564, 46576, size=len(products))
products

,ProductName,Description,UnitPrice,ProductID
0,Lamb Rack Frenched Australian,Muffin - Carrot Individual Wrap,$31.96,37536
1,Wine - Ej Gallo Sierra Valley,Vanilla Beans,$41.80,31632
2,Lid Coffee Cup 8oz Blk,Beer - Upper Canada Lager,$205.51,42836
3,"Asparagus - White, Fresh",Spinach - Packaged,$37.50,31800
4,"Mop Head - Cotton, 24 Oz",NaN,$213.22,45081
...,...,...,...,...
295,Coffee Cup 12oz 5342cd,Bread - Raisin Walnut Oval,$94.98,14124
296,Aromat Spice / Seasoning,Instant Coffee,$126.08,25904
297,Cornstarch,Canadian Emmenthal,$199.63,28810
298,"Napkin - Cocktail,beige 2 - Ply",Chevril,$133.28,23374


In [58]:
products['ProductID'].nunique()


300

In [59]:
products['UnitPrice'] = pd.to_numeric(products['UnitPrice'].str.replace('$',''))

products

,ProductName,Description,UnitPrice,ProductID
0,Lamb Rack Frenched Australian,Muffin - Carrot Individual Wrap,31.96,37536
1,Wine - Ej Gallo Sierra Valley,Vanilla Beans,41.80,31632
2,Lid Coffee Cup 8oz Blk,Beer - Upper Canada Lager,205.51,42836
3,"Asparagus - White, Fresh",Spinach - Packaged,37.50,31800
4,"Mop Head - Cotton, 24 Oz",NaN,213.22,45081
...,...,...,...,...
295,Coffee Cup 12oz 5342cd,Bread - Raisin Walnut Oval,94.98,14124
296,Aromat Spice / Seasoning,Instant Coffee,126.08,25904
297,Cornstarch,Canadian Emmenthal,199.63,28810
298,"Napkin - Cocktail,beige 2 - Ply",Chevril,133.28,23374


In [60]:
food_map = {
    'ProductCategory': [
        'Fruits', 'Fruits', 'Fruits',
        'Vegetables', 'Vegetables', 'Vegetables',
        'Dairy', 'Dairy', 'Dairy',
        'Bakery', 'Bakery', 'Bakery',
        'Snacks', 'Snacks', 'Snacks',
        'Beverages', 'Beverages', 'Beverages',
        'Condiments', 'Condiments',
        'Drinks', 'Drinks', 'Drinks',
        'Meat', 'Meat', 'Meat'
    ],
    'ProductSubcategory': [
        'Apples', 'Bananas', 'Oranges',
        'Carrots', 'Broccoli', 'Tomatoes',
        'Milk', 'Cheese', 'Yogurt',
        'Bread', 'Bagels', 'Muffins',
        'Chips', 'Nuts', 'Cookies',
        'Water', 'Juice', 'Soda',
        'Ketchup', 'Mustard',
        'Tea', 'Coffee', 'Soft Drinks',
        'Chicken', 'Beef', 'Pork'
    ]
}

# Create a DataFrame from the dictionary
food = pd.DataFrame(food_map)
food

,ProductCategory,ProductSubcategory
0,Fruits,Apples
1,Fruits,Bananas
2,Fruits,Oranges
3,Vegetables,Carrots
4,Vegetables,Broccoli
5,Vegetables,Tomatoes
6,Dairy,Milk
7,Dairy,Cheese
8,Dairy,Yogurt
9,Bakery,Bread


In [61]:
food1 = food.sample(n=len(products), replace=True)

products1 = pd.concat([products, food1.reset_index(drop=True)], axis=1)
products1

,ProductName,Description,UnitPrice,ProductID,ProductCategory,ProductSubcategory
0,Lamb Rack Frenched Australian,Muffin - Carrot Individual Wrap,31.96,37536,Condiments,Mustard
1,Wine - Ej Gallo Sierra Valley,Vanilla Beans,41.80,31632,Drinks,Tea
2,Lid Coffee Cup 8oz Blk,Beer - Upper Canada Lager,205.51,42836,Bakery,Bread
3,"Asparagus - White, Fresh",Spinach - Packaged,37.50,31800,Snacks,Nuts
4,"Mop Head - Cotton, 24 Oz",NaN,213.22,45081,Condiments,Mustard
...,...,...,...,...,...,...
295,Coffee Cup 12oz 5342cd,Bread - Raisin Walnut Oval,94.98,14124,Dairy,Cheese
296,Aromat Spice / Seasoning,Instant Coffee,126.08,25904,Vegetables,Carrots
297,Cornstarch,Canadian Emmenthal,199.63,28810,Dairy,Milk
298,"Napkin - Cocktail,beige 2 - Ply",Chevril,133.28,23374,Vegetables,Carrots


In [62]:
from faker import Faker

# number of brands to generate
brand_num = 50

# Create a Faker object
fake = Faker()

#create a function which will generate Brand name

def generate_brand_name():
    brand = fake.company()
    return brand
    
# generate a list with brand names
brands_list = [generate_brand_name() for _ in range(brand_num)]

# Create the pandas DataFrame
brands = pd.DataFrame({
    "Brand": brands_list
})

brands

,Brand
0,Patterson and Sons
1,"Gonzalez, Bailey and Turner"
2,"Heath, Haynes and Reed"
3,"Ward, Murphy and Austin"
4,Wood-Schultz
5,Moore-Hoffman
6,Lee Inc
7,Cohen-Wright
8,Schmidt-Gross
9,Lin Group


In [63]:
brands1 = brands.sample(n=len(products1), replace=True)

products2 = pd.concat([products1, brands1.reset_index(drop=True)], axis=1)

products2

,ProductName,Description,UnitPrice,ProductID,ProductCategory,ProductSubcategory,Brand
0,Lamb Rack Frenched Australian,Muffin - Carrot Individual Wrap,31.96,37536,Condiments,Mustard,Lee Inc
1,Wine - Ej Gallo Sierra Valley,Vanilla Beans,41.80,31632,Drinks,Tea,Patterson and Sons
2,Lid Coffee Cup 8oz Blk,Beer - Upper Canada Lager,205.51,42836,Bakery,Bread,Wood-Schultz
3,"Asparagus - White, Fresh",Spinach - Packaged,37.50,31800,Snacks,Nuts,Mason Ltd
4,"Mop Head - Cotton, 24 Oz",NaN,213.22,45081,Condiments,Mustard,Maxwell-Riggs
...,...,...,...,...,...,...,...
295,Coffee Cup 12oz 5342cd,Bread - Raisin Walnut Oval,94.98,14124,Dairy,Cheese,Delgado-Lindsey
296,Aromat Spice / Seasoning,Instant Coffee,126.08,25904,Vegetables,Carrots,Cervantes-Smith
297,Cornstarch,Canadian Emmenthal,199.63,28810,Dairy,Milk,Gibson-Taylor
298,"Napkin - Cocktail,beige 2 - Ply",Chevril,133.28,23374,Vegetables,Carrots,Burns-Smith


In [64]:
supp_num = 40

def generate_brand_name():
    brand = fake.company()
    return brand

# Create a dictionary with columns 'SupplierID' and 'Supplier'
data = {
    'SupplierID': np.random.randint(3489, 4536, size=supp_num),
    'Supplier': [generate_brand_name() for _ in range(supp_num)]
}

# Create a DataFrame from the dictionary
suppliers = pd.DataFrame(data)
suppliers

,SupplierID,Supplier
0,3641,"Wright, Willis and Chase"
1,4042,Norton Ltd
2,3975,"Marks, Durham and Smith"
3,4059,"Wallace, Harris and Walter"
4,4117,"Villa, Morris and Holland"
5,4249,Austin-Crawford
6,4197,"Brennan, Allen and Owen"
7,4326,Snyder-Baker
8,3663,Campos Ltd
9,4053,Davis-Johnson


In [65]:
suppliers.nunique()

SupplierID    40
Supplier      40
dtype: int64

In [66]:
suppliers1 = suppliers.sample(n=len(products2), replace=True)

products3 = pd.concat([products2, suppliers1.reset_index(drop=True)], axis=1)
products3

,ProductName,Description,UnitPrice,ProductID,ProductCategory,ProductSubcategory,Brand,SupplierID,Supplier
0,Lamb Rack Frenched Australian,Muffin - Carrot Individual Wrap,31.96,37536,Condiments,Mustard,Lee Inc,4510,"Jackson, Robinson and Baker"
1,Wine - Ej Gallo Sierra Valley,Vanilla Beans,41.80,31632,Drinks,Tea,Patterson and Sons,4055,Miller LLC
2,Lid Coffee Cup 8oz Blk,Beer - Upper Canada Lager,205.51,42836,Bakery,Bread,Wood-Schultz,4197,"Brennan, Allen and Owen"
3,"Asparagus - White, Fresh",Spinach - Packaged,37.50,31800,Snacks,Nuts,Mason Ltd,3735,Green PLC
4,"Mop Head - Cotton, 24 Oz",NaN,213.22,45081,Condiments,Mustard,Maxwell-Riggs,4053,Davis-Johnson
...,...,...,...,...,...,...,...,...,...
295,Coffee Cup 12oz 5342cd,Bread - Raisin Walnut Oval,94.98,14124,Dairy,Cheese,Delgado-Lindsey,4455,"Frazier, Smith and Rivera"
296,Aromat Spice / Seasoning,Instant Coffee,126.08,25904,Vegetables,Carrots,Cervantes-Smith,3975,"Marks, Durham and Smith"
297,Cornstarch,Canadian Emmenthal,199.63,28810,Dairy,Milk,Gibson-Taylor,3640,Evans-Hughes
298,"Napkin - Cocktail,beige 2 - Ply",Chevril,133.28,23374,Vegetables,Carrots,Burns-Smith,3640,Evans-Hughes


In [67]:
products3['ProductID'].nunique()

300

In [68]:
products3.nunique()

ProductName           285
Description           269
UnitPrice             299
ProductID             300
ProductCategory         9
ProductSubcategory     26
Brand                  50
SupplierID             40
Supplier               40
dtype: int64

In [69]:
products4 = products3.drop_duplicates()
products4.nunique()

ProductName           285
Description           269
UnitPrice             299
ProductID             300
ProductCategory         9
ProductSubcategory     26
Brand                  50
SupplierID             40
Supplier               40
dtype: int64

In [70]:
unique_combinations = products4[['ProductName', 'Description']].apply(tuple, axis=1).nunique()
print(unique_combinations)


300


In [71]:
df5

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,PostalCode,Country,StoreRegion,StoreCity,StoreAddress,CourierID,CourierName,CourierPhone,DeliveryCompanyID,DeliveryCompany
0,6253,2023-01-18 23:32:43,856,Horace,Crocket,1975-02-09,Male,135-942-6936,hcrocketnr@wunderground.com,less 25000,...,248559,UK,East of England,Luton,"Canona, Follard street, 64",85991,Alisa Ivins,694-378-6329,2860,"Cummings, Mitchell and Reynolds"
1,4685,2020-02-02 01:55:10,56,Dorthea,Brewster,1969-12-13,Female,925-496-5933,dbrewster1j@about.com,26000-50000,...,890689,UK,London,London,"Green Hills, Pine Road, 321",54743,Andrea Melluish,432-853-7949,3747,"Schaden, Beahan and Vandervort"
2,1732,2022-05-07 05:40:59,207,Darcie,Laidel,1999-07-07,Agender,750-497-1391,dlaidel5q@over-blog.com,26000-50000,...,890689,UK,London,London,"Green Hills, Pine Road, 321",36371,Antoinette Rappaport,928-696-6516,4469,Lakin Inc
3,4743,2022-09-20 17:48:16,105,Morey,Tincey,1992-05-22,Male,549-717-4109,mtincey2w@sina.com.cn,less 25000,...,867618,Spain,Madrid,Madrid,"City Center, Birch Road, 876",50088,Patty Brocking,581-863-1371,4921,Stracke-Jones
4,4522,2023-03-25 13:55:07,547,Dinah,Popland,1995-10-06,Female,796-135-4782,dpoplandf6@zimbio.com,less 25000,...,208002,UK,London,London,"Mountain Valley, Cedar Lane, 987",56218,Wilhelmine Bennough,863-331-1608,4362,"Purdy, McCullough and Kuhn"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5735,2020-08-28 20:55:21,937,Jarad,Watsham,1993-06-19,Male,793-530-2724,jwatshamq0@wikipedia.org,26000-50000,...,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654",57924,Hardy Ravenscraft,720-593-9368,5447,Leuschke and Sons
9996,5192,2020-06-17 18:14:47,601,Dyann,Lawrie,1970-05-03,Female,382-826-7050,dlawriego@washington.edu,more than 50000,...,663477,Spain,Madrid,Madrid,"Ocean View, Palm Street, 654",23931,Hetti Jimmison,279-935-4171,4889,Goyette Group
9997,5391,2022-11-02 05:19:17,607,Barty,Blodgett,1972-05-20,Male,945-792-5398,bblodgettgu@kickstarter.com,more than 50000,...,623103,Spain,Andalucia,Malaga,"New Town, Elm Street, 123",64574,Cletis Littlekit,688-939-5620,3955,Brown-Padberg
9998,861,2022-03-06 23:06:21,887,Livy,Chazier,1992-07-02,Female,227-914-8662,lchazierom@shinystat.com,less 25000,...,806129,UK,London,London,"West Side, Oak Avenue, 789",24078,Editha Tamblyn,387-587-2856,5473,Little LLC


In [72]:
# we have 10000 unique InvooiceNo
#which invoice No will contain n-InvoiceLine 
# each InvoiceLine = one line in the bill

In [73]:
# generate invoice lines
inv_line = 100000

df6 = df5.sample(n=inv_line, replace=True)

df6.count()

InvoiceNo              100000
InvoiceDate            100000
CustomerID             100000
CustomerName           100000
CustomerSurname        100000
CustomerBirthday       100000
CustomerGender         100000
CustomerPhone          100000
CustomerEmail          100000
CustomerIncome         100000
DiscountID             100000
DiscountName           100000
DiscountPercentage     100000
DiscountDescription    100000
DiscountLimit          100000
DiscountAdd            100000
DeliveryType           100000
DeliveryID             100000
StoreCode              100000
StoreType              100000
PostalCode             100000
Country                100000
StoreRegion            100000
StoreCity              100000
StoreAddress           100000
CourierID              100000
CourierName            100000
CourierPhone           100000
DeliveryCompanyID      100000
DeliveryCompany        100000
dtype: int64

In [74]:
#df6['InvoiceLine'] = range(1, len(df6) + 1)
df6['InvoiceLine'] = range(100000, 100000 + len(df6))

df6['InvoiceLine'].nunique()

100000

In [75]:
df6

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,Country,StoreRegion,StoreCity,StoreAddress,CourierID,CourierName,CourierPhone,DeliveryCompanyID,DeliveryCompany,InvoiceLine
4951,9521,2021-01-01 13:37:28,1000,Jamie,Squirrel,2000-06-20,Male,811-958-8327,jsquirrelrr@cornell.edu,more than 50000,...,Spain,Madrid,Madrid,"Cagona, Ron street, 6",85991,Alisa Ivins,694-378-6329,2860,"Cummings, Mitchell and Reynolds",100000
8664,2397,2020-07-23 02:26:19,272,Hollis,MacKibbon,1970-09-11,Male,377-945-9082,hmackibbon7j@fc2.com,more than 50000,...,Spain,Andalucia,Malaga,"Sunnyville, Maple Lane, 456",62798,Simone Stieger,449-333-1751,4332,Frami-Pfannerstill,100001
2468,6740,2021-12-15 11:13:13,738,Cacilia,Maunsell,1992-10-18,Female,621-103-4352,cmaunsellkh@youtube.com,26000-50000,...,UK,London,London,"West Side, Oak Avenue, 789",36632,Morlee Pechacek,134-705-3761,2860,"Cummings, Mitchell and Reynolds",100002
6212,6782,2021-12-06 18:29:52,284,Valeda,Ginty,1997-08-12,Female,447-561-8440,vginty7v@ehow.com,more than 50000,...,Spain,Madrid,Madrid,"City Center, Birch Road, 876",28231,Ingunna Crichten,816-885-8967,5473,Little LLC,100003
4874,184,2020-12-03 22:06:09,652,Arin,McFadzean,1981-08-30,Male,283-392-6549,amcfadzeani3@acquirethisname.com,26000-50000,...,Spain,Madrid,Madrid,"Cagona, Ron street, 6",53363,Liva Burrill,383-825-2616,3955,Brown-Padberg,100004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1943,5275,2022-08-18 04:59:15,987,Ernest,Proback,1986-07-20,Genderfluid,978-900-3150,eprobackre@hhs.gov,26000-50000,...,UK,East of England,Luton,"East Rogerberg, Gentk Road, 576",25776,Stearne Kubicki,298-541-3939,2860,"Cummings, Mitchell and Reynolds",199995
54,3033,2022-04-19 19:14:10,913,Morna,Firebrace,1999-05-01,Female,288-656-1504,mfirebracepc@cafepress.com,more than 50000,...,UK,London,London,"Mountain Valley, Cedar Lane, 987",84791,Jasmina Dutnell,472-753-1414,3870,Towne-Osinski,199996
5864,4770,2022-03-19 04:27:21,726,Goddart,Whittlesey,1987-03-14,Male,262-149-1821,gwhittleseyk5@flickr.com,less 25000,...,UK,East of England,Luton,"Riverfront, Willow Avenue, 234",83356,Maggie Noore,906-158-7150,4332,Frami-Pfannerstill,199997
1912,1023,2023-03-15 08:56:19,979,Vale,Klass,1972-11-25,Female,324-146-0850,vklassr6@go.com,more than 50000,...,UK,East of England,Luton,"East Rogerberg, Gentk Road, 576",88889,Zack Antonowicz,775-602-2334,5473,Little LLC,199998


In [76]:
products5 = products3.sample(n=len(df6), replace=True)

df7 = pd.concat([df6, products5.reset_index(drop=True)], axis=1)
print(df7)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [77]:
# As an error occured --> Reset the index of df6 before concatenation
df6.reset_index(drop=True, inplace=True)

In [78]:
df7 = pd.concat([df6, products5.reset_index(drop=True)], axis=1)
print(df7)

       InvoiceNo         InvoiceDate  CustomerID CustomerName CustomerSurname  \
0           9521 2021-01-01 13:37:28        1000        Jamie        Squirrel   
1           2397 2020-07-23 02:26:19         272       Hollis       MacKibbon   
2           6740 2021-12-15 11:13:13         738      Cacilia        Maunsell   
3           6782 2021-12-06 18:29:52         284       Valeda           Ginty   
4            184 2020-12-03 22:06:09         652         Arin       McFadzean   
...          ...                 ...         ...          ...             ...   
99995       5275 2022-08-18 04:59:15         987       Ernest         Proback   
99996       3033 2022-04-19 19:14:10         913        Morna       Firebrace   
99997       4770 2022-03-19 04:27:21         726      Goddart      Whittlesey   
99998       1023 2023-03-15 08:56:19         979         Vale           Klass   
99999       8432 2021-09-09 02:49:33         785      Juliane         Steinor   

      CustomerBirthday Cust

In [79]:
# add random product quantity for each line
df7['Quantity'] = np.random.randint(1, 6, size=len(df6))
df7

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,ProductName,Description,UnitPrice,ProductID,ProductCategory,ProductSubcategory,Brand,SupplierID,Supplier,Quantity
0,9521,2021-01-01 13:37:28,1000,Jamie,Squirrel,2000-06-20,Male,811-958-8327,jsquirrelrr@cornell.edu,more than 50000,...,"Chocolate - Pistoles, White",Rolled Oats,47.69,17276,Beverages,Water,Oliver-Miller,3545,"Taylor, Parsons and Hudson",1
1,2397,2020-07-23 02:26:19,272,Hollis,MacKibbon,1970-09-11,Male,377-945-9082,hmackibbon7j@fc2.com,more than 50000,...,Grapefruit - Pink,Shallots,120.40,40332,Fruits,Oranges,Howard PLC,3735,Green PLC,1
2,6740,2021-12-15 11:13:13,738,Cacilia,Maunsell,1992-10-18,Female,621-103-4352,cmaunsellkh@youtube.com,26000-50000,...,"Beets - Candy Cane, Organic",Lettuce - Curly Endive,34.73,20606,Meat,Beef,"Ward, Murphy and Austin",4059,"Wallace, Harris and Walter",4
3,6782,2021-12-06 18:29:52,284,Valeda,Ginty,1997-08-12,Female,447-561-8440,vginty7v@ehow.com,more than 50000,...,Melon - Honey Dew,"Wine - Red, Gallo, Merlot",285.85,15408,Vegetables,Tomatoes,Green LLC,4281,Scott and Sons,4
4,184,2020-12-03 22:06:09,652,Arin,McFadzean,1981-08-30,Male,283-392-6549,amcfadzeani3@acquirethisname.com,26000-50000,...,Sorrel - Fresh,NaN,91.81,16437,Drinks,Soft Drinks,Matthews-Long,4015,Martin Inc,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5275,2022-08-18 04:59:15,987,Ernest,Proback,1986-07-20,Genderfluid,978-900-3150,eprobackre@hhs.gov,26000-50000,...,Puree - Raspberry,Lemon Pepper,5.65,16052,Drinks,Coffee,Lee Inc,4117,"Villa, Morris and Holland",1
99996,3033,2022-04-19 19:14:10,913,Morna,Firebrace,1999-05-01,Female,288-656-1504,mfirebracepc@cafepress.com,more than 50000,...,Bread - Triangle White,Cheese - Provolone,246.90,34378,Fruits,Oranges,Bailey-Castro,3655,Jones Inc,1
99997,4770,2022-03-19 04:27:21,726,Goddart,Whittlesey,1987-03-14,Male,262-149-1821,gwhittleseyk5@flickr.com,less 25000,...,Chicken - Whole Roasting,Apple - Macintosh,218.43,23204,Meat,Beef,Bailey-Castro,4015,Martin Inc,5
99998,1023,2023-03-15 08:56:19,979,Vale,Klass,1972-11-25,Female,324-146-0850,vklassr6@go.com,more than 50000,...,Alize Gold Passion,"Fish - Halibut, Cold Smoked",83.20,27269,Dairy,Milk,Maxwell-Riggs,4117,"Villa, Morris and Holland",2


In [80]:
# then we will create calculated column --> it is more convinient to sort df
df7.sort_values(by='InvoiceNo', ascending=False, inplace=True)
df7.head(15)


,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,ProductName,Description,UnitPrice,ProductID,ProductCategory,ProductSubcategory,Brand,SupplierID,Supplier,Quantity
63962,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,"Salmon - Whole, 4 - 6 Pounds",Pepper - Chilli Seeds Mild,81.42,22585,Vegetables,Tomatoes,"Underwood, Reynolds and Cook",4173,Boyd Inc,1
7846,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Chinese Lemon Pork,Puree - Kiwi,172.67,44050,Drinks,Coffee,Cervantes-Smith,4281,Scott and Sons,2
91298,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,"Schnappes - Peach, Walkers",Yogurt - French Vanilla,250.36,41679,Dairy,Milk,"Haas, Clark and Cook",3559,Lopez and Sons,1
96968,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Scallops - In Shell,"Pork - Ham, Virginia",177.28,8542,Meat,Pork,"Hill, Jackson and Martinez",4326,Snyder-Baker,2
12010,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Bag Stand,Cocoa Powder - Natural,50.00,38605,Dairy,Milk,Cortez PLC,4042,Norton Ltd,4
44474,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,"Napkin - Cocktail,beige 2 - Ply",Chevril,133.28,23374,Vegetables,Carrots,Burns-Smith,3640,Evans-Hughes,4
66997,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Tart - Lemon,Ecolab - Power Fusion,263.85,9085,Drinks,Soft Drinks,"Heath, Haynes and Reed",4515,James-Jones,5
43197,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Kiwi,"Sprite, Diet - 355ml",178.57,5125,Drinks,Tea,"Combs, Thompson and Miller",4307,"Huffman, Valentine and Key",4
36640,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,"Juice - Apple, 500 Ml",Nut - Macadamia,26.50,24163,Snacks,Nuts,Lee Inc,4455,"Frazier, Smith and Rivera",1
20085,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Ecolab - Ster Bac,Mace,50.15,18837,Fruits,Bananas,Jones Group,4015,Martin Inc,2


In [81]:
# add TotalQuantity column = quantity of products in thw whole bill
df7['TotalQuantity'] = df7.groupby('InvoiceNo')['Quantity'].transform('sum')
df7.head(15)

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,Description,UnitPrice,ProductID,ProductCategory,ProductSubcategory,Brand,SupplierID,Supplier,Quantity,TotalQuantity
63962,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Pepper - Chilli Seeds Mild,81.42,22585,Vegetables,Tomatoes,"Underwood, Reynolds and Cook",4173,Boyd Inc,1,40
7846,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Puree - Kiwi,172.67,44050,Drinks,Coffee,Cervantes-Smith,4281,Scott and Sons,2,40
91298,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Yogurt - French Vanilla,250.36,41679,Dairy,Milk,"Haas, Clark and Cook",3559,Lopez and Sons,1,40
96968,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,"Pork - Ham, Virginia",177.28,8542,Meat,Pork,"Hill, Jackson and Martinez",4326,Snyder-Baker,2,40
12010,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Cocoa Powder - Natural,50.00,38605,Dairy,Milk,Cortez PLC,4042,Norton Ltd,4,40
44474,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Chevril,133.28,23374,Vegetables,Carrots,Burns-Smith,3640,Evans-Hughes,4,40
66997,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Ecolab - Power Fusion,263.85,9085,Drinks,Soft Drinks,"Heath, Haynes and Reed",4515,James-Jones,5,40
43197,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,"Sprite, Diet - 355ml",178.57,5125,Drinks,Tea,"Combs, Thompson and Miller",4307,"Huffman, Valentine and Key",4,40
36640,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Nut - Macadamia,26.50,24163,Snacks,Nuts,Lee Inc,4455,"Frazier, Smith and Rivera",1,40
20085,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,Mace,50.15,18837,Fruits,Bananas,Jones Group,4015,Martin Inc,2,40


In [82]:
# add LineCost column = quantity of a particular product in one line in the particular bill
df7['LineCost'] = df7['Quantity'] * df7['UnitPrice']
df7.head(15)

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,UnitPrice,ProductID,ProductCategory,ProductSubcategory,Brand,SupplierID,Supplier,Quantity,TotalQuantity,LineCost
63962,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,81.42,22585,Vegetables,Tomatoes,"Underwood, Reynolds and Cook",4173,Boyd Inc,1,40,81.42
7846,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,172.67,44050,Drinks,Coffee,Cervantes-Smith,4281,Scott and Sons,2,40,345.34
91298,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,250.36,41679,Dairy,Milk,"Haas, Clark and Cook",3559,Lopez and Sons,1,40,250.36
96968,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,177.28,8542,Meat,Pork,"Hill, Jackson and Martinez",4326,Snyder-Baker,2,40,354.56
12010,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,50.00,38605,Dairy,Milk,Cortez PLC,4042,Norton Ltd,4,40,200.00
44474,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,133.28,23374,Vegetables,Carrots,Burns-Smith,3640,Evans-Hughes,4,40,533.12
66997,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,263.85,9085,Drinks,Soft Drinks,"Heath, Haynes and Reed",4515,James-Jones,5,40,1319.25
43197,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,178.57,5125,Drinks,Tea,"Combs, Thompson and Miller",4307,"Huffman, Valentine and Key",4,40,714.28
36640,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,26.50,24163,Snacks,Nuts,Lee Inc,4455,"Frazier, Smith and Rivera",1,40,26.50
20085,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,50.15,18837,Fruits,Bananas,Jones Group,4015,Martin Inc,2,40,100.30


In [83]:
print(df7)

       InvoiceNo         InvoiceDate  CustomerID CustomerName CustomerSurname  \
63962      10000 2023-06-26 06:12:58          78      Baryram         Rapport   
7846       10000 2023-06-26 06:12:58          78      Baryram         Rapport   
91298      10000 2023-06-26 06:12:58          78      Baryram         Rapport   
96968      10000 2023-06-26 06:12:58          78      Baryram         Rapport   
12010      10000 2023-06-26 06:12:58          78      Baryram         Rapport   
...          ...                 ...         ...          ...             ...   
34063          1 2023-04-13 06:53:01         364        Filip         Balcers   
85005          1 2023-04-13 06:53:01         364        Filip         Balcers   
73606          1 2023-04-13 06:53:01         364        Filip         Balcers   
54976          1 2023-04-13 06:53:01         364        Filip         Balcers   
39446          1 2023-04-13 06:53:01         364        Filip         Balcers   

      CustomerBirthday Cust

In [84]:
# add column TotalAmount = sum of LineCost for a particular bill
df7['TotalAmount'] = df7.groupby('InvoiceNo')['LineCost'].transform('sum')
df7.head(15)

,InvoiceNo,InvoiceDate,CustomerID,CustomerName,CustomerSurname,CustomerBirthday,CustomerGender,CustomerPhone,CustomerEmail,CustomerIncome,...,ProductID,ProductCategory,ProductSubcategory,Brand,SupplierID,Supplier,Quantity,TotalQuantity,LineCost,TotalAmount
63962,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,22585,Vegetables,Tomatoes,"Underwood, Reynolds and Cook",4173,Boyd Inc,1,40,81.42,6130.92
7846,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,44050,Drinks,Coffee,Cervantes-Smith,4281,Scott and Sons,2,40,345.34,6130.92
91298,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,41679,Dairy,Milk,"Haas, Clark and Cook",3559,Lopez and Sons,1,40,250.36,6130.92
96968,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,8542,Meat,Pork,"Hill, Jackson and Martinez",4326,Snyder-Baker,2,40,354.56,6130.92
12010,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,38605,Dairy,Milk,Cortez PLC,4042,Norton Ltd,4,40,200.00,6130.92
44474,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,23374,Vegetables,Carrots,Burns-Smith,3640,Evans-Hughes,4,40,533.12,6130.92
66997,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,9085,Drinks,Soft Drinks,"Heath, Haynes and Reed",4515,James-Jones,5,40,1319.25,6130.92
43197,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,5125,Drinks,Tea,"Combs, Thompson and Miller",4307,"Huffman, Valentine and Key",4,40,714.28,6130.92
36640,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,24163,Snacks,Nuts,Lee Inc,4455,"Frazier, Smith and Rivera",1,40,26.50,6130.92
20085,10000,2023-06-26 06:12:58,78,Baryram,Rapport,1961-08-16,Male,872-342-2392,brapport25@unicef.org,26000-50000,...,18837,Fruits,Bananas,Jones Group,4015,Martin Inc,2,40,100.30,6130.92


In [85]:
# add column TotalAmount_After_Discount = sum of LineCost for a particular bill - Discoun
df7['TotalAmount_After_Discount'] = round(df7['TotalAmount'] * (100 - df7['DiscountPercentage']) / 100, 2)


In [86]:
df7.loc[df7['InvoiceNo'] == 10000][['InvoiceNo','Quantity','UnitPrice', 'InvoiceLine', 'LineCost', 'TotalQuantity', 'TotalAmount', 'DiscountPercentage','TotalAmount_After_Discount']]

,InvoiceNo,Quantity,UnitPrice,InvoiceLine,LineCost,TotalQuantity,TotalAmount,DiscountPercentage,TotalAmount_After_Discount
63962,10000,1,81.42,163962,81.42,40,6130.92,11,5456.52
7846,10000,2,172.67,107846,345.34,40,6130.92,11,5456.52
91298,10000,1,250.36,191298,250.36,40,6130.92,11,5456.52
96968,10000,2,177.28,196968,354.56,40,6130.92,11,5456.52
12010,10000,4,50.00,112010,200.00,40,6130.92,11,5456.52
44474,10000,4,133.28,144474,533.12,40,6130.92,11,5456.52
66997,10000,5,263.85,166997,1319.25,40,6130.92,11,5456.52
43197,10000,4,178.57,143197,714.28,40,6130.92,11,5456.52
36640,10000,1,26.50,136640,26.50,40,6130.92,11,5456.52
20085,10000,2,50.15,120085,100.30,40,6130.92,11,5456.52


In [87]:
# change columns order as it will be in external table in Postgres Database
change_col_order = ['DiscountPercentage',
  'DiscountName',
  'DeliveryCompanyID',
  'DeliveryCompany',
  'ProductID' ,
  'ProductName',
  'StoreType',
  'PostalCode',
  'StoreCode',
  'StoreAddress',
  'InvoiceNo',
  'DeliveryType',
  'DiscountID',
  'Description',
  'Quantity',
  'InvoiceDate',
  'UnitPrice',
  'CustomerID',
  'Country',
  'StoreCity',
  'StoreRegion',
  'CustomerName',
  'CustomerSurname',
  'CustomerBirthday',
  'CustomerGender',
  'CustomerPhone',
  'CustomerEmail',
  'CustomerIncome',
  'ProductCategory',
  'ProductSubcategory',
  'Brand',
  'SupplierID',
  'Supplier',
  'DeliveryID',
  'CourierID',
  'CourierName',
  'CourierPhone',
  'DiscountDescription',
  'DiscountLimit',
  'DiscountAdd',
  'InvoiceLine',
  'LineCost',
  'TotalQuantity',
  'TotalAmount',
  'TotalAmount_After_Discount']


df7 = df7.reindex(columns=change_col_order)
df7.head()

,DiscountPercentage,DiscountName,DeliveryCompanyID,DeliveryCompany,ProductID,ProductName,StoreType,PostalCode,StoreCode,StoreAddress,...,CourierName,CourierPhone,DiscountDescription,DiscountLimit,DiscountAdd,InvoiceLine,LineCost,TotalQuantity,TotalAmount,TotalAmount_After_Discount
63962,11,Holiday Discount,2860,"Cummings, Mitchell and Reynolds",22585,"Salmon - Whole, 4 - 6 Pounds",online,261048,WY6587-g,"East Rogerberg, Gentk Road, 576",...,Emilio Vampouille,832-779-3343,Special pricing for the spring season.,Only work days,Limited quantity available.,163962,81.42,40,6130.92,5456.52
7846,11,Holiday Discount,2860,"Cummings, Mitchell and Reynolds",44050,Chinese Lemon Pork,online,261048,WY6587-g,"East Rogerberg, Gentk Road, 576",...,Emilio Vampouille,832-779-3343,Special pricing for the spring season.,Only work days,Limited quantity available.,107846,345.34,40,6130.92,5456.52
91298,11,Holiday Discount,2860,"Cummings, Mitchell and Reynolds",41679,"Schnappes - Peach, Walkers",online,261048,WY6587-g,"East Rogerberg, Gentk Road, 576",...,Emilio Vampouille,832-779-3343,Special pricing for the spring season.,Only work days,Limited quantity available.,191298,250.36,40,6130.92,5456.52
96968,11,Holiday Discount,2860,"Cummings, Mitchell and Reynolds",8542,Scallops - In Shell,online,261048,WY6587-g,"East Rogerberg, Gentk Road, 576",...,Emilio Vampouille,832-779-3343,Special pricing for the spring season.,Only work days,Limited quantity available.,196968,354.56,40,6130.92,5456.52
12010,11,Holiday Discount,2860,"Cummings, Mitchell and Reynolds",38605,Bag Stand,online,261048,WY6587-g,"East Rogerberg, Gentk Road, 576",...,Emilio Vampouille,832-779-3343,Special pricing for the spring season.,Only work days,Limited quantity available.,112010,200.00,40,6130.92,5456.52


In [88]:
df7.to_csv('dataset_Europe.csv', index=False)